you may run these commands in terminal

particularly in this notebook functions to run streamlit and FastAPI have been designed using subprocess.run()

Both functions initiate in sepearte thread, hence no blocking to main thread


In [1]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz --no-check-certificate
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok ~/DigitalMarketingAIOptimization

--2024-09-11 07:58:42--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 9085090 (8.7M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.5’

ngrok-v3-stable-lin 100%[===================>]   8.66M  4.59MB/s    in 1.9s    

2024-09-11 07:58:45 (4.59 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.5’ saved [9085090/9085090]

ngrok
mv: 'ngrok' and '/home/sagemaker-user/DigitalMarketingAIOptimization/ngrok' are the same file


In [2]:
!pip install pyngrok


In [3]:
import subprocess
import threading
import time
from pyngrok import ngrok
import os
import signal
import socket
import sys

In [4]:
# Set the correct project root directory
project_root = '/home/sagemaker-user/DigitalMarketingAIOptimization'
sys.path.insert(0, project_root)

In [5]:
def get_free_port(start_port):
    port = start_port
    while True:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(('', port))
                return port
            except OSError:
                port += 1

def run_streamlit(port):
    try:
        env = os.environ.copy()
        env['PYTHONPATH'] = f"{project_root}:{env.get('PYTHONPATH', '')}"
        app_path = os.path.join(project_root, "app", "dashboard", "app.py")
        subprocess.run(["streamlit", "run", app_path, "--server.port", str(port)], env=env, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running Streamlit: {e}")

def run_fastapi(port):
    try:
        env = os.environ.copy()
        env['PYTHONPATH'] = f"{project_root}:{env.get('PYTHONPATH', '')}"
        api_path = os.path.join(project_root, "app", "api", "main.py")
        subprocess.run(["uvicorn", "app.api.main:app", "--host", "0.0.0.0", "--port", str(port)], env=env)
    except subprocess.CalledProcessError as e:
        print(f"Error running FastAPI: {e}")

In [6]:
# Get free ports
streamlit_port = get_free_port(8501)
fastapi_port = get_free_port(8000)

print(f"Using port {streamlit_port} for Streamlit")
print(f"Using port {fastapi_port} for FastAPI")

Using port 8502 for Streamlit
Using port 8000 for FastAPI


In [7]:
# Start Streamlit and FastAPI in separate threads
streamlit_thread = threading.Thread(target=run_streamlit, args=(streamlit_port,))
fastapi_thread = threading.Thread(target=run_fastapi, args=(fastapi_port,))

streamlit_thread.start()
fastapi_thread.start()

# Wait for the servers to start
time.sleep(10)




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8502
  Network URL: http://169.255.255.2:8502
  External URL: http://43.204.56.180:8502



INFO:     Started server process [27082]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


In [8]:
print("Current working directory:", os.getcwd())
print("\nContents of the data directory:")
for root, dirs, files in os.walk('data'):
    for file in files:
        print(os.path.join(root, file))

Current working directory: /home/sagemaker-user/DigitalMarketingAIOptimization

Contents of the data directory:
data/raw/googleads-performance.csv
data/raw/microsoftads-performance.csv
data/raw/metaads-performance.csv
data/raw/website-landings.csv
data/raw/.ipynb_checkpoints/metaads-performance-checkpoint.csv
data/raw/.ipynb_checkpoints/googleads-performance-checkpoint.csv
data/raw/.ipynb_checkpoints/microsoftads-performance-checkpoint.csv
data/processed/combined_ad_data.csv
data/processed/feature_engineered_data.csv
data/processed/featured_data_with_anomalies.csv
data/processed/.ipynb_checkpoints/combined_ad_data-checkpoint.csv
data/synthetic/synthetic_ad_data.csv


In [9]:
ngrok.set_auth_token("2ltmnY5NoRmDJqAwZN2Uvt88VMG_3UJr4P85WwUYcGJYUxToM")

In [10]:
streamlit_tunnel = ngrok.connect(streamlit_port)
fastapi_tunnel = ngrok.connect(fastapi_port)

print(f"Streamlit URL: {streamlit_tunnel.public_url}")
print(f"FastAPI URL: {fastapi_tunnel.public_url}")

Streamlit URL: https://b77c-43-204-56-180.ngrok-free.app
FastAPI URL: https://51ab-43-204-56-180.ngrok-free.app


t=2024-09-11T07:59:05+0000 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"


In [ ]:
# Keep the notebook running to maintain the tunnels
try:
    while True:
        time.sleep(60)
        print("Tunnels are active...")
except KeyboardInterrupt:
    print("Stopping tunnels and servers...")
    ngrok.disconnect(streamlit_tunnel.public_url)
    ngrok.disconnect(fastapi_tunnel.public_url)
    # Attempt to stop the Streamlit and FastAPI servers
    os.kill(os.getpid(), signal.SIGINT)


2024-09-11 07:59:31.265 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert Timestamp('2024-03-31 18:33:10.352504832') with type Timestamp: tried to convert to int64", 'Conversion failed for column Date with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-09-11 07:59:31.717 Uncaught app exception
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 590, in code_to_exec
    exec(code, module.__dict__)
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 127, in <module>
    main()
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 28, in main
    display_data_overview(data)
  File "/home/sage

Tunnels are active...


2024-09-11 08:00:28.676 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert Timestamp('2024-03-31 18:33:10.352504832') with type Timestamp: tried to convert to int64", 'Conversion failed for column Date with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-09-11 08:00:28.747 Uncaught app exception
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 590, in code_to_exec
    exec(code, module.__dict__)
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 127, in <module>
    main()
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 28, in main
    display_data_overview(data)
  File "/home/sage

Tunnels are active...
Tunnels are active...


2024-09-11 08:02:04.888 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert Timestamp('2024-03-31 18:33:10.352504832') with type Timestamp: tried to convert to int64", 'Conversion failed for column Date with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-09-11 08:02:04.947 Uncaught app exception
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 590, in code_to_exec
    exec(code, module.__dict__)
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 127, in <module>
    main()
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 28, in main
    display_data_overview(data)
  File "/home/sage

Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...


2024-09-11 08:06:22.773 Serialization of dataframe to Arrow table was unsuccessful due to: ("Could not convert Timestamp('2024-03-31 18:33:10.352504832') with type Timestamp: tried to convert to int64", 'Conversion failed for column Date with type object'). Applying automatic fixes for column types to make the dataframe Arrow-compatible.
2024-09-11 08:06:22.825 Uncaught app exception
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/exec_code.py", line 88, in exec_func_with_error_handling
    result = func()
  File "/opt/conda/lib/python3.10/site-packages/streamlit/runtime/scriptrunner/script_runner.py", line 590, in code_to_exec
    exec(code, module.__dict__)
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 127, in <module>
    main()
  File "/home/sagemaker-user/DigitalMarketingAIOptimization/app/dashboard/app.py", line 28, in main
    display_data_overview(data)
  File "/home/sage

Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...
Tunnels are active...


In [ ]:
ngrok.kill()

In [ ]:
print("Tunnels closed. Restart your Jupyter kernel to ensure all processes are stopped.")